### BUSINESS UNDERSTANDING
Buiding a model that classifies whether an email is a spam or not

## DATA UNDERSTANDING

In [86]:
#Importing libraries
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.collocations import *
import string
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize import RegexpTokenizer

In [87]:
#loading our dataset
spam=pd.read_csv('Spam.csv')
pd.set_option('display.max_colwidth',100)
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [88]:
#preveiwing our data set
spam.tail()

,Category,Message
5567,spam,This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other suggestions?"
5570,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week ...
5571,ham,Rofl. Its true to its name


In [89]:
spam.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [90]:
spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


### DATA CLEANING

In [91]:
spam

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other suggestions?"
5570,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week ...


In [92]:
#checking for missing values
spam.isna().sum()

Category    0
Message     0
dtype: int64

In [93]:
spam.duplicated().sum()

415

In [94]:
# A function that checks for duplicates and drops them 
def check_dup(df):
    print(f'There are {df.duplicated().sum()} duplicates')

    """dropping the dupicates """
    df.drop_duplicates(inplace=True)
    
    """Checking whether the duplicates have  been dropped"""
    print(f'After droping ,there are {df.duplicated().sum()} duplicates')

In [95]:
check_dup(spam)

There are 415 duplicates
After droping ,there are 0 duplicates


#### Cleaning the texts using NLTK

In [96]:
spam

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other suggestions?"
5570,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week ...


In [97]:
#lowercasing the texts
def lowercase(df,column):
    "Converting dataframe to lowercase"
    df[column]= df[column].apply(lambda x: str(x).lower())
    return df


In [98]:
lowercase(spam,'Message')


,Category,Message
0,ham,"go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there g..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive ...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives around here though"
...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u. u have won the £750 pound prize. 2 claim is easy...
5568,ham,will ü b going to esplanade fr home?
5569,ham,"pity, * was in mood for that. so...any other suggestions?"
5570,ham,the guy did some bitching but i acted like i'd be interested in buying something else next week ...


In [106]:
#removing punctuation in the texts
def remove_punct(text):
    text_punct= "".join([c for c in text if c not in string.punctuation])
    return text_punct





In [107]:

spam['punc_message']=spam['Message'].apply(lambda x:remove_punct(x))
spam.head()

,Category,Message,punc_message
0,ham,"go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there g...",go until jurong point crazy available only in bugis n great world la e buffet cine there got amo...
1,ham,ok lar... joking wif u oni...,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive ...,free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive e...
3,ham,u dun say so early hor... u c already then say...,u dun say so early hor u c already then say
4,ham,"nah i don't think he goes to usf, he lives around here though",nah i dont think he goes to usf he lives around here though
